Package the files as an application (.tar)

In [2]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating style_transfer.egg-info
writing style_transfer.egg-info/PKG-INFO
writing top-level names to style_transfer.egg-info/top_level.txt
writing dependency_links to style_transfer.egg-info/dependency_links.txt
writing manifest file 'style_transfer.egg-info/SOURCES.txt'
reading manifest file 'style_transfer.egg-info/SOURCES.txt'
writing manifest file 'style_transfer.egg-info/SOURCES.txt'

running check


creating style_transfer-0.1
creating style_transfer-0.1/style_transfer.egg-info
copying files to style_transfer-0.1...
copying predictor.py -> style_transfer-0.1
copying setup.py -> style_transfer-0.1
copying style_transfer.egg-info/PKG-INFO -> style_transfer-0.1/style_transfer.egg-info
copying style_transfer.egg-info/SOURCES.txt -> style_transfer-0.1/style_transfer.egg-info
copying style_transfer.egg-info/dependency_links.txt -> style_transfer-0.1/style_transfer.egg-info
copying style_transfer.egg-info/top_level.txt -> style_transfer-0.1/style_transfer.

Send the application package to the bucket

In [4]:
!gsutil cp dist/style_transfer-0.1.tar.gz gs://luchos_own_personal_cool_bucket/staging_style_transfer/

Copying file://dist/style_transfer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.7 KiB/  2.7 KiB]                                                
Operation completed over 1 objects/2.7 KiB.                                      


Create a model: separates containeraized GCP resources for deployment

In [1]:
# The code below can be replaced by going through the console in "models" section
#!gcloud ai-platform models create "style-transfer"

"Fill" the model: indicate the actual version and application. I did it through the console, but here's the code

In [17]:
MODEL_DIR="gs://luchos_own_personal_cool_bucket/staging_style_transfer/" #staging_style_transfer/"
VERSION_NAME="0.1"
MODEL_NAME="style_transfer"
CUSTOM_CODE_PATH="gs://luchos_own_personal_cool_bucket/staging_style_transfer/style_transfer-0.1.tar.gz	"
PREDICTOR_CLASS="MyPredictor.merge_content_and_style"

In [18]:
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --deploymentUri $MODEL_DIR \
  --runtime-version=1.14 \
  --python-version=3.5\
  --package-uris=$CUSTOM_CODE_PATH\
  --prediction-class=$PREDICTOR_CLASS

ERROR: (gcloud.beta.ai-platform.versions.create) unrecognized arguments:
  --deploymentUri (did you mean '--document'?)
  gs://luchos_own_personal_cool_bucket/staging_style_transfer/
  To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS


Finally, notice that "runtime-version" defines the environment for deployment. At the moment (still under beta), there's no tensorflow 2.0 (bummer!). One could probably use the old code (coursera implementation), and try and deploy it... BUT... althought there's a 1.14 environment for that in deployment, the "notebooks" version in the VMs is not compatible, so cannot run it and test it in the VM before deploying. (probably more tweaking could do it, but that will have to wait until a next iteration or project).

Now, wait for 2.0 to be av